## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 3s 0us/step
(50000, 32, 32, 3)


In [0]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [5]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32,32,3)))
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(momentum=0.99,epsilon=0.001))

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization(momentum=0.99,epsilon=0.001))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu'))
#output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
50000/50000 [==============================] - 8s 164us/step - loss: 1.2709 - acc: 0.5599
Epoch 2/100
50000/50000 [==============================] - 5s 92us/step - loss: 0.8203 - acc: 0.7103
Epoch 3/100
50000/50000 [==============================] - 5s 92us/step - loss: 0.6024 - acc: 0.7886
Epoch 4/100
50000/50000 [==============================] - 5s 91us/step - loss: 0.4213 - acc: 0.8533
Epoch 5/100
50000/50000 [==============================] - 4s 89us/step - loss: 0.2875 - acc: 0.8995
Epoch 6/100
50000/50000 [==============================] - 4s 89us/step - loss: 0.1930 - acc: 0.9344
Epoch 7/100
50000/50000 [==============================] - 5s 91us/step - loss: 0.1489 - acc: 0.9495
Epoch 8/100
50000/50000 [==============================] - 5s 91us/step - loss: 0.1215 - acc: 0.9575
Epoch 9/100
50000/50000 [==============================] - 5s 91us/step - loss: 0

In [7]:
score = classifier.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 3.2398884048461913
Test accuracy: 0.6819


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[6.2090662e-12, 1.9891789e-16, 2.3239147e-08, 9.9996305e-01,
        3.6978345e-05, 1.7450230e-15, 1.6958310e-11, 3.1573543e-14,
        5.4984867e-11, 5.4222687e-16]], dtype=float32)